# Übung 6: Transfer learning

Neuronal Netze benötigen oft eine große Menge an Trainingsdaten, damit es nicht zu overfitting kommt. Transfer learning erlaubt es, mit relativ geringen Datenmenge dennoch erfolgreiche große Netze zu trainieren. Dabei verwendet man ein bereits auf einen anderen Datensatz (z.b. ImageNet) vortrainiertes Netzwerk, und ersetzt nur das letzte Layer durch ein neues. In dieser Übung geht es darum, ein Netzwerk für die Erkennung von Geparden und Leoparden in der freien Wildbahn zu trainineren. 

## Daten laden

Lade die Daten hier herunter: http://tonic.imp.fu-berlin.de/cv_data/data.tar.gz

(die Daten liegen auch entpackt hier: http://tonic.imp.fu-berlin.de/cv_data/data/)

Die Daten wurde bereits in ein Trainings- und Validierungsset geteilt. Die Ordnerstruktur ist wie bei vielen Bildklassifierungsdatensetzen so aufgebaut. Es gibt zwei Unterordner für die Trainings- und Validierunsdaten. In diesen Ordnern liegen dann jeweils alle Bilder von einer Klasse in einem Unterordner mit dem Namen der Klasse.

Ein Beispiel: Die Trainingsbilder für die Klasse "cheetah" liegen in dem Unterordner train/cheetah

Diese Orderstruktur wird auch von dem in keras enhaltenen ImageDataGenerator unterstützt.

In [15]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import resnet50
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix

In [2]:
batch_size = 32
image_input_size = (32, 32)
data_path = 'data/'

In [3]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')

izw_classes = ('unknown', 'cheetah', 'leopard')

generator = ImageDataGenerator(horizontal_flip=True)
val_generator = ImageDataGenerator(horizontal_flip=False)

train_gen = generator.flow_from_directory(
    train_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)

val_gen = val_generator.flow_from_directory(
    val_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)

Found 17857 images belonging to 3 classes.
Found 1915 images belonging to 3 classes.


## Training ohne transfer learning

Trainiere zuerst ein kleines Classifer-Netzwerk ohne transfer learning. Falls du keine Grafikkarte hast, solltest du nicht die volle Auflösung (siehe Variable image_input_size) verwenden, da das Training sonst zu lange dauert. Eine Bildgröße von 32x32 Pixeln wäre zum Beispiel möglich.

In [50]:
model = Sequential([
    Conv2D(4, (3, 3), activation='relu', input_shape=image_input_size + (3,)),
    MaxPooling2D(strides=2),
    Dropout(0.4),

    Conv2D(8, (3, 3), activation='relu'),
    MaxPooling2D(strides=2),
    Dropout(0.4),

    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(strides=2),
    
    Flatten(),
    Dropout(0.4),
    Dense(1000, activation='relu'),
    Dense(len(izw_classes), activation='softmax')
])

model.compile(optimizer=SGD(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
model.fit_generator(train_gen, epochs=100, validation_data=val_gen, shuffle=True, verbose=1)

Epoch 1/100
559/559 [==============================] - 141s 253ms/step - loss: 7.4433 - acc: 0.4504 - val_loss: 1.7336 - val_acc: 0.7295
Epoch 2/100
559/559 [==============================] - 130s 232ms/step - loss: 5.2160 - acc: 0.5778 - val_loss: 1.4088 - val_acc: 0.7050
Epoch 3/100
559/559 [==============================] - 155s 277ms/step - loss: 4.6746 - acc: 0.5689 - val_loss: 1.1418 - val_acc: 0.6637
Epoch 4/100
559/559 [==============================] - 157s 282ms/step - loss: 3.7309 - acc: 0.5309 - val_loss: 0.9648 - val_acc: 0.6809
Epoch 5/100
559/559 [==============================] - 156s 279ms/step - loss: 2.7235 - acc: 0.5205 - val_loss: 0.9349 - val_acc: 0.6230
Epoch 6/100
559/559 [==============================] - 178s 319ms/step - loss: 2.1583 - acc: 0.5106 - val_loss: 0.8415 - val_acc: 0.7097
Epoch 7/100
559/559 [==============================] - 213s 381ms/step - loss: 1.7607 - acc: 0.5116 - val_loss: 0.8457 - val_acc: 0.6867
Epoch 8/100
559/559 [====================

In [55]:
evaluation = model.evaluate_generator(val_gen, steps=len(val_gen))
print('loss: {:.4f}, acc: {:.4f}'.format(evaluation[0], evaluation[1]))

loss: 0.9105, acc: 0.7191


Erstelle eine Confusion matrix basierend auf den Ausgaben des Netzes für die Validierungsdaten und berechne den ROC AUC für die Klasse cheetah. Du kannst hierfür optional die scikit-learn Bibliothek verwenden.

In [57]:
prediction = model.predict_generator(val_gen, steps=len(val_gen), verbose=1)

60/60 [==============================] - 16s 270ms/step


In [69]:
cm = confusion_matrix(val_gen.classes, np.argmax(prediction, axis=1))
display(val_gen.class_indices)
display(cm)

{'cheetah': 1, 'leopard': 2, 'unknown': 0}

array([[  79,  285,    0],
       [ 266, 1174,    0],
       [  20,   91,    0]])

## Pretrained network

Lade nun ein auf Imagenet vortrainiertes Netzwerk und klassifiziere damit die Validierungsdaten. Eine Anleitung für keras findest du hier: https://keras.io/applications

Du kannst selber entscheiden, welche Netzwerkarchitektur du verwendest.

In [290]:
pre_image_input_size = (224,224)

pre_train_gen = generator.flow_from_directory(
    train_data_path,
    target_size=pre_image_input_size,
    classes=izw_classes,
    batch_size=batch_size
)

pre_val_gen = val_generator.flow_from_directory(
    val_data_path,
    target_size=pre_image_input_size,
    classes=izw_classes,
    batch_size=batch_size
)

Found 17857 images belonging to 3 classes.
Found 1915 images belonging to 3 classes.


In [296]:
pre_model = resnet50.ResNet50(weights='imagenet')

In [297]:
pre_predictions = pre_model.predict_generator(pre_val_gen, steps=len(pre_val_gen), verbose=1)

60/60 [==============================] - 678s 11s/step


In [298]:
def printTop3(predictions):
    print(("{:20}{:15}\t" * (3)).format("Class 1", "Score 1", "Class 2", "Score 2", "Class 3", "Score 3"))
    for ((_, class1, score1),
         (_, class2, score2),
         (_, class3, score3)) in predictions:
        data = (class1[:19], score1,
                class2[:19], score2,
                class3[:19], score3)
        print(("{:20}{:.12}\t" * (3)).format(*data))

In [299]:
display(pre_predictions)
pre_decodedPredictions = resnet50.decode_predictions(pre_predictions, top=3)
printTop3(pre_decodedPredictions)

array([[1.0109814e-06, 3.3007746e-05, 1.9052401e-05, ..., 6.8476584e-07,
        1.6734755e-05, 1.4160064e-02],
       [1.2569237e-08, 3.0987007e-07, 2.7192182e-06, ..., 1.4136625e-06,
        3.6770550e-07, 2.7442850e-08],
       [9.8323130e-07, 5.4954212e-06, 3.8383255e-04, ..., 2.0046177e-06,
        2.4610878e-05, 2.9327408e-05],
       ...,
       [8.9246705e-06, 2.8517523e-05, 1.7795200e-04, ..., 7.2618539e-04,
        1.1817818e-04, 7.5894138e-07],
       [4.4902504e-06, 2.0052306e-04, 1.2218258e-04, ..., 4.0232730e-06,
        3.8435384e-05, 1.8529037e-02],
       [4.6115329e-05, 1.0674555e-04, 1.1316130e-04, ..., 8.1183226e-04,
        3.4583016e-05, 1.0554364e-02]], dtype=float32)

Class 1             Score 1        	Class 2             Score 2        	Class 3             Score 3        	
fountain            0.701757252216	bathtub             0.0329484827816	tub                 0.0203170310706	
mosquito_net        0.889322817326	window_screen       0.0764325261116	swing               0.0183949414641	
fountain            0.943378448486	geyser              0.0100959856063	breakwater          0.00624788925052	
cheetah             0.966064453125	fountain            0.00794915296137	leopard             0.00303715560585	
fountain            0.89999204874	geyser              0.0645431652665	breakwater          0.00547980237752	
cheetah             0.433352082968	fountain            0.329920649529	jaguar              0.0254553817213	
fountain            0.278256982565	park_bench          0.0602722615004	broom               0.0362431965768	
fountain            0.820691049099	zebra               0.0186572372913	book_jacket         0.0135150253773	
fountain            0.196

fountain            0.444177210331	cheetah             0.0410466231406	park_bench          0.0304591916502	
dalmatian           0.83484774828	cheetah             0.161762997508	zebra               0.00195506191812	
fountain            0.994055747986	geyser              0.00251624896191	breakwater          0.00089956773445	
fountain            0.517253756523	cheetah             0.0834002718329	pickelhaube         0.0452063307166	
park_bench          0.141715079546	megalith            0.08056525141	sloth_bear          0.0619997121394	
fountain            0.996037185192	cheetah             0.000690356304403	geyser              0.000438519287854	
car_mirror          0.137657865882	park_bench          0.107501737773	fountain            0.106238260865	
cheetah             0.869481503963	dalmatian           0.060216512531	fountain            0.053445674479	
fountain            0.998520195484	geyser              0.000503015064169	cliff               6.72972819302e-05	
fountain            0.986

Da der ImageNet-Datensatz auch die Klassen cheetah und leopard enthält, können wir sogar ohne transfer learning das vortrainierte Netzwerk evaluieren. Interpretiere alle Klassen außer cheetah und leopard als unknown und berechne wie im vorherigen Schritt die Confusion matrix und den ROC AUC score für die Klasse cheetah.

In [304]:
pre_class_prediction = [1 if x[0][1] == izw_classes[1] else (2 if x[0][1] == izw_classes[2] else 0) for x in pre_decodedPredictions]
pre_cm = confusion_matrix(pre_val_gen.classes, pre_class_prediction)
display(pre_val_gen.class_indices)
display(pre_cm)

{'cheetah': 1, 'leopard': 2, 'unknown': 0}

array([[ 272,   92,    0],
       [1028,  410,    2],
       [  84,   26,    1]])

## Transfer learning

Das vortrainierte Netzwerk kann nun mit unseren Daten weitertrainiert werden. Ersetze dafür das letzte Layer in dem Netzwerk mit einem Dense Layer mit 3 Ausgaben für unsere Klassen cheetah, leopard und unknown. Du kannst selbst entscheiden, ob du nun das komplette Netzwerk mit trainierst oder nur das neu eingefügte, letzte Layer.

Auch hierfür kannst du dich wieder an der keras Anleitung orientieren: https://keras.io/applications

In [301]:
pre_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_52 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_52[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [302]:
transfer_model = Model(pre_model.input, pre_model.output)
transfer_model.layers.pop()

for layer in transfer_model.layers:
    layer.trainable = False

output = transfer_model.layers[-1].output
output = Dense(3, activation='softmax')(output)
transfer_model = Model(transfer_model.input, output)
transfer_model.compile(optimizer=SGD(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
transfer_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_52 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_52[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Evaluiere das so trainierte Netzwerk wie in den letzten beiden Aufgaben.

In [307]:
transfer_model.fit_generator(pre_train_gen, epochs=1, validation_data=pre_val_gen, shuffle=True, verbose=1)

Epoch 1/1
559/559 [==============================] - 5887s 11s/step - loss: 0.9958 - acc: 0.4622 - val_loss: 1.3155 - val_acc: 0.3138


In [308]:
transfer_evaluation = transfer_model.evaluate_generator(pre_val_gen)
print('loss: {:.4f}, acc: {:.4f}'.format(transfer_evaluation[0], transfer_evaluation[1]))

loss: 1.3155, acc: 0.3138


In [309]:
transfer_prediction = transfer_model.predict_generator(pre_val_gen, steps=len(pre_val_gen), verbose=1)

60/60 [==============================] - 599s 10s/step


In [310]:
transfer_cm = confusion_matrix(pre_val_gen.classes, np.argmax(transfer_prediction, axis=1))
display(pre_val_gen.class_indices)
display(transfer_cm)

{'cheetah': 1, 'leopard': 2, 'unknown': 0}

array([[184, 139,  41],
       [722, 579, 139],
       [ 54,  42,  15]])

## Auswertung

Beschreibe kurz qualitativ die Resultate. Wie unterscheiden sich die trainierten Netzwerke, zum Beispiel im Bezug auf die Genauigkeit oder die Laufzeit? Welche Entscheidungen musstest du bei der Erfüllung der Aufgaben treffen und warum hast du dich für den von dir gewählten Weg entschieden?